# DataprocSparkConnect SQL Magic Demo

This notebook demonstrates how to use the Spark SQL magic commands with DataprocSparkSession.

## Setup

First, let's create a DataprocSparkSession and load the magic extension.

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from pyspark.sql import SparkSession

# Create a DataprocSparkSession (or use regular SparkSession for local testing)
# For DataprocSparkSession:
# spark = DataprocSparkSession.builder.getOrCreate()

# For local testing with regular SparkSession:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [ ]:
# Load the Spark SQL magic extension
%load_ext sparksql_magic

## Configuration

You can configure the default row display limit.

In [ ]:
# Check current configuration
%config

In [ ]:
# Set the default row limit to 30
%config SparkSql.limit=30

## Create Sample Data

Let's create some sample data to work with.

In [ ]:
# Create a sample DataFrame
data = [
    (1, "Alice", 25, "Engineering"),
    (2, "Bob", 30, "Marketing"),
    (3, "Charlie", 35, "Engineering"),
    (4, "David", 28, "Sales"),
    (5, "Eve", 32, "Marketing"),
    (6, "Frank", 29, "Engineering"),
    (7, "Grace", 31, "Sales"),
    (8, "Henry", 27, "Marketing")
]

columns = ["id", "name", "age", "department"]
employees_df = spark.createDataFrame(data, columns)
employees_df.createOrReplaceTempView("employees")

print("Sample data created successfully!")

## Basic SQL Query

Execute a simple SQL query.

In [ ]:
%%sparksql
SELECT * FROM employees

## Query with Filtering

In [ ]:
%%sparksql
SELECT name, age, department 
FROM employees 
WHERE age > 30
ORDER BY age DESC

## Capture Result in Variable

Store the query result in a variable for further processing.

In [ ]:
%%sparksql engineers_df
SELECT * FROM employees WHERE department = 'Engineering'

In [ ]:
# Now we can use the engineers_df variable
print(f"Number of engineers: {engineers_df.count()}")
engineers_df.show()

## Create Temporary View

Create a new temporary view from a query result.

In [ ]:
%%sparksql --view senior_employees
SELECT * FROM employees WHERE age >= 30

In [ ]:
%%sparksql
SELECT department, COUNT(*) as count 
FROM senior_employees 
GROUP BY department

## Cache DataFrame

Cache a DataFrame for better performance on repeated queries.

In [ ]:
%%sparksql --cache --view cached_data result_df
SELECT 
    department,
    AVG(age) as avg_age,
    COUNT(*) as employee_count
FROM employees
GROUP BY department

## Variable Interpolation

Use Python variables in SQL queries.

In [ ]:
min_age = 28
target_department = "Engineering"

In [ ]:
%%sparksql
SELECT * 
FROM employees 
WHERE age >= {min_age} 
  AND department = '{target_department}'

## Override Display Limit

Override the default display limit for a specific query.

In [ ]:
%%sparksql --limit 3
SELECT * FROM employees ORDER BY age

## Eager Caching

Cache with eager loading for immediate execution.

In [ ]:
%%sparksql --eager dept_summary
SELECT 
    department,
    MIN(age) as min_age,
    MAX(age) as max_age,
    AVG(age) as avg_age
FROM employees
GROUP BY department

## Complex Query Example

A more complex query with window functions.

In [ ]:
%%sparksql
WITH ranked_employees AS (
    SELECT 
        name,
        age,
        department,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY age DESC) as rank_in_dept
    FROM employees
)
SELECT * 
FROM ranked_employees 
WHERE rank_in_dept <= 2
ORDER BY department, rank_in_dept

## Cleanup

Stop the Spark session when done.

In [ ]:
# spark.stop()